In [55]:
### Google Secret Manager Client Instalation


# !pip install google-cloud-secret-manager
# Install gcloud: https://cloud.google.com/sdk/docs/install#windows
# Setup the credential: gcloud auth application-default login

In [88]:
# !pip install aioredis==1.3.1

In [58]:
import json
from google.cloud import secretmanager
import redis
import asyncio
import aioredis

import numpy as np

from tqdm.notebook import tqdm

In [59]:
def get_secret_value(project_id = "652914548272", secret_id = "redis", key = "redis-test"):
    from google.cloud import secretmanager

    # Create the Secret Manager client.
    client = secretmanager.SecretManagerServiceClient()
    
    name = f"projects/{project_id}/secrets/{secret_id}/versions/latest"

    # Access the secret version.
    response = client.access_secret_version(request={"name": name})

    payload = response.payload.data.decode("UTF-8")
    configs = eval(payload)

    config_key = configs[key]

    return config_key


In [87]:
# Redis on Redis Cloud (30mb)
config_redis = get_secret_value(key = "redis-test")

In [62]:
# Redis on Virtual Machine (GCP Compute Engine)
# config_redis = {
#     'host': '34.122.96.184',
#     'port': 6379,
#     'password':"redis-stack"
# }

In [63]:
r = redis.Redis(
  host=config_redis["host"],
  port=config_redis["port"],
  password=config_redis["password"]
)

# Load Index Json

In [65]:
with open("index.json", 'r+') as f:
    data_json = json.load(f)

/Users/alim/miniconda3/envs/main/lib/python3.11/json/decoder.py:353: RuntimeWarning: coroutine 'batch_push' was never awaited
  obj, end = self.scan_once(s, idx)
Task was destroyed but it is pending!
task: <Task pending name='Task-13' coro=<RedisConnection._read_data() running at /Users/alim/miniconda3/envs/main/lib/python3.11/site-packages/aioredis/connection.py:186> wait_for=<Future pending cb=[Task.task_wakeup()]> cb=[RedisConnection.__init__.<locals>.<lambda>() at /Users/alim/miniconda3/envs/main/lib/python3.11/site-packages/aioredis/connection.py:168]>


# Push Index to Redis

In [70]:
# r.set("document_size", str(data_json['document_size']))

In [71]:
r.get('document_size')

b"{'0': 5000}"

In [72]:
# r.set("doc_ids_list", str(data_json['doc_ids_list']))

In [74]:
# r.get('doc_ids_list')

In [75]:
# # Store term positions in documents
# for term in tqdm(data_json.keys()):
#     if term not in ['document_size', 'doc_ids_list']:
#         r.set(term, str(data_json['term'])) #38944/39310

# Push Index to Redis in batches

In [83]:
def process_dict_in_batches(input_dict, batch_size):
    keys = list(input_dict.keys())
    num_keys = len(keys)
    batches = []
    for i in range(0, num_keys, batch_size):
        batch_keys = keys[i:i+batch_size]
        batch = {key: str(input_dict[key]) for key in batch_keys}
        batches.append(batch)
    return batches

In [86]:
minibatch = 15
batches = process_dict_in_batches(data_json, minibatch)

In [85]:
async def set_data(redis, key, value):
    await redis.set(key, value)

async def batch_push():
    # Define Redis server configuration
    redis_config = {
        'address': (config_redis["host"], config_redis["port"]),  # IP address and port
        'password': config_redis["password"]  # Redis password
    }
    
    # Connect to Redis
    redis = await aioredis.create_redis_pool(**redis_config)

    # Define keys and values to set
    for batch in tqdm(batches):
        # Perform SET operations in parallel
        tasks = [set_data(redis, key, value) for key, value in batch.items()]
        await asyncio.gather(*tasks)

    # Close Redis connection
    redis.close()
    await redis.wait_closed()

# Run the asyncio event loop
# asyncio.run(batch_push()) # use in a script

# await batch_push() # use in a notebook

# Retrieve value from Redis

In [76]:
def get_val(r, key):
    value = r.get(key)
    value = value.decode()
    value = eval(value)
    return value

In [77]:
# get_val(r, 'doc_ids_list')

In [78]:
terms = list(data_json.keys())

In [81]:
term = terms[-21]
val = get_val(r, term)

In [82]:
val

{'10037': [911, 912]}

# Create a new format index

In [68]:
# data = {
#     "document_size": data_json['document_size'],
#     "doc_ids_list": data_json['doc_ids_list']
# }

# data['terms'] = {}

In [69]:
# for keyword in data_json.keys():
#     if keyword not in ['document_size', 'doc_ids_list']:
#         data['terms'][keyword] = data_json[keyword]